In [ ]:
import pandas as pd
import numpy as np
import difflib

Caricamento dei dataset

In [ ]:
spotify = pd.read_csv("songs_final.csv")
stream = pd.read_csv("StreamCount_sistemato.csv")
yt = pd.read_csv("Youtube_Sistemato.csv")

In [ ]:
spotify.drop('Unnamed: 0' , axis=1, inplace=True)
stream.drop('Unnamed: 0' , axis=1, inplace=True)
yt.drop(['Unnamed: 0' ] , axis=1, inplace=True)

## Data Quality

In [ ]:
type(yt['Views'][0])

In [ ]:
type(stream['Stream'][0])

Notiamo come il valore delle visualizazzioni su youtube sia in formato diverso da quello del conteggio delle stream su spotify, per cui modifichiamo questo ultimo

In [ ]:
stream['Stream'] = stream['Stream'].str.replace(".", "")
stream[stream['Stream']!='Null']['Stream'].astype(np.float64)
stream

### Cerchiamo le righe doppie nel Dataset del conteggio delle stream di spotify, e teniamo solo quelle con il numero di stream maggiore

In [ ]:
doppi = stream[stream.duplicated(subset=(['Artist' , 'Track', 'Album']) , keep=False)==True]
massimi = doppi.groupby(['Artist' , 'Track', 'Album'],as_index=False)['Stream'].max()

art = massimi['Artist']
trac =massimi['Track']
stre = massimi['Stream']
alb = massimi['Album']


for i in range(len(art)):
    stream.drop(stream[(stream['Artist']==art[i]) & (stream['Track']==trac[i]) & (stream['Album']==alb[i]) & (stream['Stream']!=stre[i])].index,
             axis=0, inplace=True)

stream

### Uniamo il Dataset di spotify API con quello di Youtube

In [ ]:
sp_yt= pd.merge(spotify , yt , left_index=True, right_index=True , copy=False)
sp_yt.drop(['Artist_y' , 'Track_y'], axis=1, inplace=True)
sp_yt = sp_yt.rename(columns={'Artist_x':'Artist', 'Url_x':'Url_spotify', 'Track_x':'Track',
                        'Url_y':'Url_youtube'})
sp_yt

### Unisco il dataset ``sp_yt`` con il dataset delle stream

In [ ]:
finale = pd.merge( sp_yt , stream , on=['Artist' , 'Track', 'Album'], how='left' )
finale

### Restano delle righe doppie, le eliminiamo

In [ ]:
finale[finale.duplicated(subset=(['Artist' , 'Track', 'Album']) , keep=False)]

In [ ]:
finale.drop_duplicates(subset=['Artist' , 'Track', 'Album'], inplace=True)
finale.drop(['Url' , 'Track_Api'] , axis=1, inplace=True)
finale.reset_index(inplace=True, drop=True)
finale

In [ ]:
finale.to_csv("Dataset_finale.csv")

# Completezza 

In [ ]:
finale.isnull().sum()